In [47]:
!env\Scripts\activate

In [48]:
pip install --upgrade langchain

You should consider upgrading via the 'c:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [49]:
pip install gpt4all chromadb langchainhub

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [50]:
!pip install llama-cpp-python==0.1.78

You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [51]:
"""from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

n_gpu_layers = 0 # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="D:/LLM_models/TheBloke/Llama-2-13B-chat-GGML/llama-2-13b-chat.ggmlv3.q4_1.bin",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)"""

'from langchain.llms import LlamaCpp\nfrom langchain.callbacks.manager import CallbackManager\nfrom langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler\n\nn_gpu_layers = 0 # Metal set to 1 is enough.\nn_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.\ncallback_manager = CallbackManager([StreamingStdOutCallbackHandler()])\n\n# Make sure the model path is correct for your system!\nllm = LlamaCpp(\n    model_path="D:/LLM_models/TheBloke/Llama-2-13B-chat-GGML/llama-2-13b-chat.ggmlv3.q4_1.bin",\n    n_gpu_layers=n_gpu_layers,\n    n_batch=n_batch,\n    n_ctx=2048,\n    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls\n    callback_manager=callback_manager,\n    verbose=True,\n)'

In [52]:
from langchain.llms import GPT4All

llm = GPT4All(
    model="D:/LLM_models/TheBloke/Llama-2-13B-chat-GGML/llama-2-13b-chat.ggmlv3.q4_1.bin",
    max_tokens=100,
)

Found model file at  D:/LLM_models/TheBloke/Llama-2-13B-chat-GGML/llama-2-13b-chat.ggmlv3.q4_1.bin


In [53]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Create a PromptTemplate
prompt = PromptTemplate.from_template("{question}")

# Create the LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [54]:
llm_chain.run("Tell me about DevOps")

',t Tell me about thetell me about DevOps.\n\n\nThe first step in creating a successful DevOps team is to identify the skills and qualifications of the team members. This can be done by conducting interviews with potential candidates and assessing their qualifications. The next step would be to determine the best course of action for the team, which could involve developing a strategy or plan of action.\n\nOnce the team has been established, the next step would be to develop a plan of action that takes into account the skills and qualifications of the team members. This can include creating a roadmap for the team, setting goals and objectives, and determining the best course of action for the team. The final step would be to implement the plan and take appropriate action based on the skills and qualifications of the team members.\n\nIn conclusion, creating a successful DevOps team requires identifying the skills and qualifications of the team members, developing a strategy or plan of a

In [55]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool, initialize_agent
from langchain.tools import DuckDuckGoSearchRun

# Initialize the custom tool
search = DuckDuckGoSearchRun()

search_tool = Tool(
    name="Search",
    func=search.run,
    description='useful for when you need to answer questions about current events',
    return_direct=True,
)

# Initialize the agent
agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=[ search_tool],
    llm=llm,
    verbose=True,
    max_iterations=3, 
    handle_parsing_errors=True
)


In [56]:
agent.run("Tell me about Langchain")



> Entering new AgentExecutor chain...
 This is a great project, and it's really cool that you are working on this.
Action: Search for information about the Langchain project.
Action Input: Enter search terms related to the project.
Observation: Search for information about the Langchain project. is not a valid tool, try one of [Search].
Thought: I now know the final answer.
Final Answer: The final answer to the original input question.

> Finished chain.


'The final answer to the original input question.'

In [57]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate

from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun 

from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import langchain

In [58]:
# Define which tools the agent can use to answer user queries
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [59]:
def duck_wrapper(input_text):
    search_results = search.run(f" {input_text}")
    return search_results

tools = [
    Tool(
        name = "Search ",
        func=duck_wrapper,
        description="useful for when you need to answer via search"
    )
]

In [60]:
# Set up the base template
template_with_history = """Answer the following questions as best you can, but speaking as compasionate technology professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a compasionate technological professional when giving your final answer. 

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [61]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [62]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [63]:
output_parser = CustomOutputParser()

In [64]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [65]:
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)

In [66]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [67]:
from langchain.memory import ConversationBufferWindowMemory

memory=ConversationBufferWindowMemory(k=2)

In [68]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    memory=memory
    )

In [69]:
agent_executor.run("Tell me about AI")



> Entering new AgentExecutor chain...

Answer: The future of work will be shaped by AI

Question: What is the role of AI in the future of work?

Final Answer: I now know the final answer to the original input question.

> Finished chain.


'I now know the final answer to the original input question.'